In [11]:
!pip install tqdm

In [12]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import pickle
import os
from tqdm import tqdm
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import shutil

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ruta_csv = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/SuperEventos.csv'
df_eventos = pd.read_csv(ruta_csv)

# Convertir la columna 'inicio' a formato datetime
df_eventos['inicio'] = pd.to_datetime(df_eventos['inicio'])

# Seleccionar solo los primeros 3 registros para la prueba
df_eventos_prueba = df_eventos.head(10)

# Configuración de la API
API_KEY = 'f3fb2d3eea1f4d7e8a126d2802950cf2'
base_url = "https://api.weatherbit.io/v2.0/history/hourly"

output_dir = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/Clima/'
resultados_file = os.path.join(output_dir, "resultados_v6.pkl")
super_tabla_file = os.path.join(output_dir, "super_tabla_clima_v6.csv")

In [5]:
# Función para realizar la consulta a la API
def consultar_api(params):
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error en la consulta para ({params['lat']}, {params['lon']}, {params['start_date']} - {params['end_date']}): {response.reason}")
        return None

# Crear una lista de tareas de consulta
tareas = []
index = 0
for _, row in df_eventos.iterrows():
    longitud = row['LONGITUD']
    latitud = row['LATITUD']
    inicio = row['inicio']

    # Calcular la ventana temporal (un día antes )
    start_date = (inicio - timedelta(hours=24)).strftime('%Y-%m-%d')
    end_date = inicio.strftime('%Y-%m-%d')

    params = {
        "lon": longitud,
        "lat": latitud,
        "start_date": start_date,
        "end_date": end_date,
        "tz": "local",
        "key": API_KEY
    }
    tareas.append((params, inicio, row, index))
    index += 1

In [14]:
assert len(tareas) == len(df_eventos), "La cantidad de tareas no coincide con la cantidad de eventos."

In [15]:
to_keep = [
 #'app_temp',
 #'azimuth',
 'clouds',
 #'datetime',
 #'dewpt',
 #'dhi',
 #'dni',
 #'elev_angle',
 #'ghi',
 #'h_angle',
 #'pod',
 'precip',
 'pres',
 #'revision_status',
 'rh',
 'slp',
 #'snow',
 'solar_rad',
 'temp',
 #'timestamp_utc',
 #'ts',
 'uv',
 'vis',
 #'weather',
 #'wind_dir',
 'wind_gust_spd',
 'wind_spd',
 ]

In [17]:
if os.path.exists(resultados_file):
    with open(resultados_file, "rb") as archivo:
        resultados = pickle.load(archivo)
else:
  resultados = {}


for params, inicio, row, index in tqdm(tareas[55789:86000]):

    if index in resultados:
        continue

    respuesta = consultar_api(params)

    if respuesta and "data" in respuesta:
        # resultados[index] = []


        registros = respuesta["data"]
        h_registros = [{f"h{i}-{k}": v for k, v in registro.items() if k in to_keep} for i, registro in enumerate(registros)]
        h_registro = {k: v for d in h_registros for k, v in d.items()}

        # Almacenar la fecha y hora original del evento junto con los datos de la API
        resultados[index] = {
            "inicio_evento": inicio,  # Fecha y hora original del evento
            **h_registro,
            **row.to_dict(),
        }

        if index % 2000== 0:
            with open(resultados_file, "wb") as archivo:
                pickle.dump(resultados, archivo)
            shutil.copyfile(resultados_file, os.path.join(output_dir, f"resultados_backup.pkl"))

    else:
        print("Respuesta de la API no contiene datos o hubo un error.")
        print(params)
        break

100%|██████████| 30211/30211 [09:52<00:00, 51.00it/s]


In [18]:
# Convertir la lista de resultados a un DataFrame
df_resultados = pd.DataFrame.from_dict(resultados, orient='index')
df_resultados.to_csv(super_tabla_file, index=False, encoding='utf-8')

print("Tabla de prueba generada y almacenada como 'resultados_climaticos_prueba.csv'.")
df_resultados

Tabla de prueba generada y almacenada como 'resultados_climaticos_prueba.csv'.


,inicio_evento,h0-clouds,h0-precip,h0-pres,h0-rh,h0-slp,h0-solar_rad,h0-temp,h0-uv,h0-vis,...,NIVEL2CONDUCTOR,NIVEL3CONDUCTOR,NIVEL4CONDUCTOR,ACOMETIDACONDUCTOR,NEUTRO,CALIBRENEUTRO,CAPACITY,RESISTANCE,LONGITUD2,LATITUD2
55789,2021-04-27 19:42:07,99,14.75,980.0,96.0,1011.0,0,23.7,0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55790,2021-04-27 19:42:07,99,14.75,980.0,96.0,1011.0,0,23.7,0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55791,2021-04-27 19:42:07,99,14.75,980.0,96.0,1011.0,0,23.7,0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55792,2021-04-28 09:30:07,100,4.00,986.0,93.0,1012.0,0,23.3,0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55793,2021-04-28 07:50:07,100,4.00,986.0,93.0,1012.0,0,23.3,0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85995,2022-08-01 04:57:05,100,1.00,913.0,89.0,1016.0,0,17.0,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85996,2022-08-01 00:31:01,100,1.00,798.0,86.0,1019.0,0,13.0,0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85997,2022-08-01 08:01:55,100,0.75,903.0,89.0,1016.0,0,17.0,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85998,2022-08-01 08:01:55,100,0.75,903.0,89.0,1016.0,0,17.0,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
frames = [

    pd.read_csv(os.path.join(output_dir, "super_tabla_clima.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v2.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v3.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v4.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v5.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v6.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v7.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v8.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v9.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v10.csv")),
    pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v11.csv")),
]
result = pd.concat(frames)

result.to_csv(os.path.join(output_dir, "super_tabla_clima_v0.csv"), index=False, encoding='utf-8')
result

<ipython-input-19-9114f57ec000>:4: DtypeWarning: Columns (312,321,322) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v2.csv")),
<ipython-input-19-9114f57ec000>:6: DtypeWarning: Columns (312) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v4.csv")),
<ipython-input-19-9114f57ec000>:7: DtypeWarning: Columns (312,321,322) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v5.csv")),
<ipython-input-19-9114f57ec000>:8: DtypeWarning: Columns (312,321,322) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(os.path.join(output_dir, "super_tabla_clima_v6.csv")),
<ipython-input-19-9114f57ec000>:9: DtypeWarning: Columns (312,321,322) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.re

,inicio_evento,h0-clouds,h0-precip,h0-pres,h0-rh,h0-slp,h0-solar_rad,h0-temp,h0-uv,h0-vis,...,NIVEL2CONDUCTOR,NIVEL3CONDUCTOR,NIVEL4CONDUCTOR,ACOMETIDACONDUCTOR,NEUTRO,CALIBRENEUTRO,CAPACITY,RESISTANCE,LONGITUD2,LATITUD2
0,2019-01-01 00:00:01,51.0,0.00,803.0,82.0,1014.0,0,12.6,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 00:00:01,51.0,0.00,803.0,82.0,1014.0,0,12.6,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 00:00:01,51.0,0.00,803.0,82.0,1014.0,0,12.6,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 00:00:01,51.0,0.00,803.0,82.0,1014.0,0,12.6,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 00:00:01,51.0,0.00,803.0,82.0,1014.0,0,12.6,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,2022-12-09 15:05:01,89.0,0.00,763.0,95.0,1018.0,0,13.3,0.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13996,2022-12-09 06:50:51,95.0,2.50,908.0,91.0,1014.0,0,17.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13997,2022-12-09 15:30:29,70.0,1.75,879.0,87.0,1015.0,0,18.5,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13998,2022-12-09 16:29:35,76.0,0.25,907.0,89.0,1015.0,0,18.9,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(tareas)

10

In [20]:
df_eventos

,evento,equipo_ope,tipo_equi_ope,cto_equi_ope,tipo_elemento,inicio,fin,duracion_h,tipo_duracion,causa,...,NIVEL2CONDUCTOR,NIVEL3CONDUCTOR,NIVEL4CONDUCTOR,ACOMETIDACONDUCTOR,NEUTRO,CALIBRENEUTRO,CAPACITY,RESISTANCE,LONGITUD2,LATITUD2
0,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 01:01:49,1.030000,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 01:09:36,1.159722,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 01:18:21,1.305556,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 02:41:00,2.683056,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 13:22:25,13.373333,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168220,8272936,M24083,interruptor,VMA23L15,13.2,2024-06-30 18:17:03,2024-06-30 19:55:11,1.635556,> 3 min,Causa desconocida,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168221,8272938,M24081,transformador,VMA23L15,TFD,2024-06-30 18:17:40,2024-06-30 20:10:52,1.886667,> 3 min,Causa desconocida,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168222,8272942,VBO23L14,interruptor,VBO23L14,13.2,2024-06-30 23:11:25,2024-06-30 23:13:14,0.030278,<= 3 min,Condiciones atmosféricas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168223,8272942,VBO23L14,interruptor,VBO23L14,13.2,2024-06-30 23:11:25,2024-06-30 23:13:33,0.035556,<= 3 min,Condiciones atmosféricas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
